# Vienna Land Use

The ..

#  (1) Land Use resampling for 100m

In [2]:
# 1. transformation of land use vector dataset to the 100m Vienna GRID:

# geometry:
import geopandas as gpd

# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")
# 2 LOAD "land uise"
shp = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\landnutzung\REALNUT2020OGD_31256.shp")

# Perform spatial union
union_result = gpd.overlay(shp_100m_GRID, shp, how='union')

# Save the result to a new shapefile
union_result.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m.shp")

print ("done")




C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\shapely\set_operations.py:76: RuntimeWarning: divide by zero encountered in difference
  return lib.difference(a, b, **kwargs)
C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\shapely\set_operations.py:76: RuntimeWarning: invalid value encountered in difference
  return lib.difference(a, b, **kwargs)
C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_26936\3323111743.py:12: UserWarning: `keep_geom_type=True` in overlay resulted in 8 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  union_result = gpd.overlay(shp_100m_GRID, shp, how='union')


done


The following script calc. the area of land use class inside every 100x100m cell. Then the land use with the larrgest area will be selected and exported to  new vector grid file:

# 2) for every 100m cell calc area of 100x100m cover in percent:

In [78]:
import pandas as pd



## Land use table with FLY_indicator:


'''
NUTZUNG_CODE	NUTZUNG_LE	NUTZUNG_L0	NUTZUNG_L1	FLY_indiciator
1	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	locker bebautes Wohn(misch)gebiet	2
2	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	Wohn(misch)gebiet mittlerer Dichte	1
3	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	dichtes Wohn(misch)gebiet	1
4	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	grossvolumiger, solidaerer Wohn(misch)bau	1
5	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	Buero- und Verwaltungsviertel	1
6	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	solitaere Handelsstrukturen	1
7	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	Geschaefts-, Kern- u. Mischgebiete	1
8	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	Mischnutzung wenig dicht	1
9	Baulandnutzung	Industrie- und Gewerbenutzung	Industrie, prod. Gewerbe, Grosshandel inkl. Lager	0
10	Baulandnutzung	soziale Infrastruktur	Kultur, Freizeit, Messe	1
11	Baulandnutzung	soziale Infrastruktur	Gesundheit und Einsatzorg.	1
12	Baulandnutzung	soziale Infrastruktur	Bildung	1
13	Baulandnutzung	soziale Infrastruktur	Sport und Bad (Indoor)	0
14	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Militaerische Anlagen	1
15	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Klaeranlage, Deponie	2
16	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Energieversorgung u. Rundfunkanlagen	0
17	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Wasserversorgung	0
18	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Transformationsflaeche, Baustelle, Materialgew.	0
19	Verkehr	Strassenraum	Strassenraum begruent	1
20	Verkehr	Strassenraum	Strassenraum unbegruent	0
21	Verkehr	weitere verkehrliche Nutzungen	Parkpluetze u. Parkhaeuser	0
22	Verkehr	weitere verkehrliche Nutzungen	Bahnhoefe und Bahnanlagen	0
23	Verkehr	weitere verkehrliche Nutzungen	Transport und Logistik inkl. Lager	0
24	Gruenlandnutzung	Erholungs- u. Freizeiteinrichtungen	Park, Gruenanlage	1
25	Gruenlandnutzung	Erholungs- u. Freizeiteinrichtungen	Sport und Bad (Outdoor), Camping	1
26	Gruenlandnutzung	Erholungs- u. Freizeiteinrichtungen	Friedhof	1
27	Gruenlandnutzung	Landwirtschaft	Acker	1
28	Gruenlandnutzung	Landwirtschaft	Weingarten	2
29	Gruenlandnutzung	Landwirtschaft	Gaertnerei, Obstplantagen	2
30	Gruenlandnutzung	Naturraum	Wald	1
31	Gruenlandnutzung	Naturraum	Wiese	1
32	Gruenlandnutzung	Gewaeser	Gewaeser inkl. Bachbett	0


'''


import pandas as pd

data2 = {
    'NUTZUNG_CODE': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
  

        'NUTZUNG_LE3': [  'locker bebautes Wohn(misch)gebiet', 
                     'Wohn(misch)gebiet mittlerer Dichte', 
                     'dichtes Wohn(misch)gebiet',
                     'grossvolumiger und solidaerer Wohn(misch)bau', 
                     'Buero- und Verwaltungsviertel', 
                     'solitaere Handelsstrukturen', 
                     'Geschaefts- u. Kern- u. Mischgebiete', 
                     'Mischnutzung wenig dicht', 
                     'Industrie prod. Gewerbe, Grosshandel inkl. Lager',
                     'Kultur und  Freizeit und  Messe', 
                     'Gesundheit und Einsatzorg', 
                     'Bildung', 
                     'Sport und Bad (Indoor)', 
                     'Militaerische Anlagen', 
                     'Klaeranlage und  Deponie', 
                     'Energieversorgung u. Rundfunkanlagen', 
                     'Wasserversorgung', 
                     'Transformationsflaeche  ,Baustelle, Materialgew.', 
                     'Strassenraum begruent', 
                     'Strassenraum unbegruent',
                     'Parkpluetze u. Parkhaeuser', 
                     'Bahnhoefe und Bahnanlagen', 
                     'Transport und Logistik inkl. Lager',
                     'Park und Gruenanlage', 
                     'Sport und Bad (Outdoor) Camping', 
                     'Friedhof', 
                     'Acker', 
                     'Weingarten',
                     'Gaertnerei und Obstplantagen', 
                     'Wald',
                     'Wiese', 
                     'Gewaeser inkl. Bachbett'],

                     'NUTZUNG_LE2': [

                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Industrie- und Gewerbenutzung',
                                'soziale Infrastruktur',
                                'soziale Infrastruktur',
                                'soziale Infrastruktur',
                                'soziale Infrastruktur',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Strassenraum',
                                'Strassenraum',
                                'weitere verkehrliche Nutzungen',
                                'weitere verkehrliche Nutzungen',
                                'weitere verkehrliche Nutzungen',
                                'Erholungs- u. Freizeiteinrichtungen',
                                'Erholungs- u. Freizeiteinrichtungen',
                                'Erholungs- u. Freizeiteinrichtungen',
                                'Landwirtschaft',
                                'Landwirtschaft',
                                'Landwirtschaft',
                                'Naturraum',
                                'Naturraum',
                                'Gewaeser'],



                       'NUTZUNG_LE1': [
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Verkehr',
                        'Verkehr',
                        'Verkehr',
                        'Verkehr',
                        'Verkehr',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung'],
                 'FLY_indiciator': [
                        '2',
                        '1',
                        '1',
                        '1',
                        '1',
                        '1',
                        '1',
                        '1',
                        '0',
                        '1',
                        '1',
                        '1',
                        '0',
                        '1',
                        '2',
                        '0',
                        '0',
                        '0',
                        '1',
                        '0',
                        '0',
                        '0',
                        '0',
                        '1',
                        '1',
                        '1',
                        '1',
                        '2',
                        '2',
                        '1',
                        '1',
                        '0']

}

df_land_use_LUT = pd.DataFrame(data2)

(df_land_use_LUT)



,NUTZUNG_CODE,NUTZUNG_LE3,NUTZUNG_LE2,NUTZUNG_LE1,FLY_indiciator
0,1,locker bebautes Wohn(misch)gebiet,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,2
1,2,Wohn(misch)gebiet mittlerer Dichte,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,1
2,3,dichtes Wohn(misch)gebiet,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,1
3,4,grossvolumiger und solidaerer Wohn(misch)bau,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,1
4,5,Buero- und Verwaltungsviertel,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
5,6,solitaere Handelsstrukturen,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
6,7,Geschaefts- u. Kern- u. Mischgebiete,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
7,8,Mischnutzung wenig dicht,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
8,9,"Industrie prod. Gewerbe, Grosshandel inkl. Lager",Industrie- und Gewerbenutzung,Baulandnutzung,0
9,10,Kultur und Freizeit und Messe,soziale Infrastruktur,Baulandnutzung,1


In [72]:
import geopandas as gpd
##Real_land_use2020_100m_test_group_by3
# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m.shp")
out_dem_vector =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m_max_land_use_in_cell.shp"

gdf = shp_100m_GRID
#print(gdf.head())


# Calculate the area for each geometry in the GeoDataFrame
gdf['area'] = gdf.geometry.area

# Print the first few rows to check the calculated areas
#print(gdf[['geometry', 'area']].head())


# Group by the attribute field and calculate aggregate statistics
grouped_stats = gdf.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()


# Group by 'GRID100mID' and find the index of the row with the maximum area for each group
##idx = gdf.groupby('GRID100mID')['area'].idxmax()




In [77]:
## filter by land luse




import pandas as pd
selected_rows = gdf[gdf['NUTZUNG_CO'] == 30]
grouped_stats_30 = selected_rows.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()
grouped_stats_30['FLY_indiciator'] = 1

grouped_stats_30

GRID100mID                          NUTZUNG_CO
EPSG31256_GRID_100m_E-10000N335000  30.0            370.597432
EPSG31256_GRID_100m_E-10000N335100  30.0           8853.027136
EPSG31256_GRID_100m_E-10000N335200  30.0           9696.248073
EPSG31256_GRID_100m_E-10000N335300  30.0           8776.315390
EPSG31256_GRID_100m_E-10000N335400  30.0          10000.000000
                                                      ...     
EPSG31256_GRID_100m_E9900N344300    30.0           3234.316662
EPSG31256_GRID_100m_E9900N345100    30.0           3516.093626
EPSG31256_GRID_100m_E9900N345200    30.0           3757.677364
EPSG31256_GRID_100m_E9900N345300    30.0           1388.749225
FLY_indiciator                                        1.000000
Name: area, Length: 13433, dtype: float64

In [120]:
type(grouped_stats_30)

pandas.core.series.Series

In [157]:
## loop by land use table:

print ("-------------------------- START:")
gdf['FLY_indiciator'] = 0
gdf['NUTZUNG_LE3'] = 'xx'
for i in range(len(df_land_use_LUT['NUTZUNG_CODE'])):
    code = df_land_use_LUT['NUTZUNG_CODE'][i]
    le3 = df_land_use_LUT['NUTZUNG_LE3'][i]
    #le2 = df_land_use_LUT['NUTZUNG_LE2'][i]
    #le1 = df_land_use_LUT['NUTZUNG_LE1'][i]
    fly_indicator = df_land_use_LUT['FLY_indiciator'][i]
    
    print ('Land_use_code   '+ str(code))
    print (le3)
    print ('Fly indicator:   '+ str(fly_indicator))

    gdf.loc[gdf['NUTZUNG_CO'] == code, 'FLY_indiciator'] = fly_indicator
    gdf.loc[gdf['NUTZUNG_CO'] == code, 'NUTZUNG_LE3'] = le3 
    selected_rows = gdf[gdf['NUTZUNG_CO'] == code]
    
    #grouped_stats_lu = selected_rows.groupby(['GRID100mID','NUTZUNG_CO','NUTZUNG_LE3','FLY_indiciator'])['area'].sum()
   
    # Dissolve the geometries based on the attribute field
    #dissolved_gdf = selected_rows.dissolve(by=['GRID100mID','NUTZUNG_CO','NUTZUNG_LE3','FLY_indiciator'])

    #gdf_to_import = gdf['GRID100mID','area']

    
    merged_gdf = gdf.merge(selected_rows, on='GRID100mID', how='left')

    print(merged_gdf.head())

    # Dissolve the geometries based on the attribute field

    #merged_gdf.fillna(0, inplace=True)
    #dissolved_gdf = merged_gdf.dissolve(by=['GRID100mID'])
    #dissolved_gdf = merged_gdf.dissolve(by=['GRID100mID','NUTZUNG_CO','NUTZUNG_LE3','FLY_indiciator'])

    


    #out_dem_vector  =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\land_use\Real_land_use2020_100m_max_land_use_in_cell_lucode_"+str(code)+".shp"
    #dissolved_gdf.to_file(out_dem_vector)
    print ("done")


    print ("--------------------------")
    
   
   
          

-------------------------- START:
Land_use_code   1
locker bebautes Wohn(misch)gebiet
Fly indicator:   2
    EAST_x   NORTH_x                          GRID100mID  OBJECTID_x BEZ_x  \
0 -11200.0  336800.0  EPSG31256_GRID_100m_E-11200N336800     26514.0    13   
1 -11200.0  336900.0  EPSG31256_GRID_100m_E-11200N336900     26514.0    13   
2 -11200.0  337000.0  EPSG31256_GRID_100m_E-11200N337000     26514.0    13   
3 -11100.0  336700.0  EPSG31256_GRID_100m_E-11100N336700     26514.0    13   
4 -11100.0  336800.0  EPSG31256_GRID_100m_E-11100N336800     26514.0    13   

  ZBEZ_x ZGEB_x     BLK_x  NUTZUNG_CO_x      NUTZUNG_LE_x  ... BLK_y  \
0   1310  13100  13100011          30.0  GrÃ¼nlandnutzung  ...   NaN   
1   1310  13100  13100011          30.0  GrÃ¼nlandnutzung  ...   NaN   
2   1310  13100  13100011          30.0  GrÃ¼nlandnutzung  ...   NaN   
3   1310  13100  13100011          30.0  GrÃ¼nlandnutzung  ...   NaN   
4   1310  13100  13100011          30.0  GrÃ¼nlandnutzung  ...   N

In [154]:
gdf_to_import

0         EPSG31256_GRID_100m_E-11200N336800
1         EPSG31256_GRID_100m_E-11200N336900
2         EPSG31256_GRID_100m_E-11200N337000
3         EPSG31256_GRID_100m_E-11100N336700
4         EPSG31256_GRID_100m_E-11100N336800
                         ...                
177940     EPSG31256_GRID_100m_E18700N353500
177941     EPSG31256_GRID_100m_E18700N353600
177942     EPSG31256_GRID_100m_E18700N353700
177943     EPSG31256_GRID_100m_E18700N353800
177944     EPSG31256_GRID_100m_E18700N353900
Name: GRID100mID, Length: 177945, dtype: object

In [145]:
dissolved_gdf = merged_gdf.dissolve(by=['GRID100mID','NUTZUNG_CO','NUTZUNG_LE3','FLY_indiciator'])

AttributeError: 'DataFrame' object has no attribute 'dissolve'

# 3) for every 100m  calc. max land use:

In [42]:
##Real_land_use2020_100m_test_group_by3
# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m.shp")
out_dem_vector =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m_max_land_use_in_cell.shp"

gdf = shp_100m_GRID
#print(gdf.head())


# Calculate the area for each geometry in the GeoDataFrame
gdf['area'] = gdf.geometry.area

# Print the first few rows to check the calculated areas
#print(gdf[['geometry', 'area']].head())


# Group by the attribute field and calculate aggregate statistics
grouped_stats = gdf.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()


# Group by 'GRID100mID' and find the index of the row with the maximum area for each group
idx = gdf.groupby('GRID100mID')['area'].idxmax()
# Use the index to select the corresponding 'land_use' value
max_land_use = gdf.loc[idx, ['GRID100mID', 'NUTZUNG_CO']]
max_land_use = max_land_use.rename(columns={'NUTZUNG_CO': 'NUTZ_MAX'})
print(max_land_use.head())
merged_gdf = gdf.merge(max_land_use, on='GRID100mID', how='left')

#print(merged_gdf.head())

# Dissolve the geometries based on the attribute field
dissolved_gdf = merged_gdf.dissolve(by=['GRID100mID','NUTZ_MAX'])



dissolved_gdf.to_file(out_dem_vector)
print ("done")



                                GRID100mID  NUTZ_MAX
151935  EPSG31256_GRID_100m_E-10000N331000       NaN
151936  EPSG31256_GRID_100m_E-10000N331100       NaN
151937  EPSG31256_GRID_100m_E-10000N331200       NaN
151938  EPSG31256_GRID_100m_E-10000N331300       NaN
151939  EPSG31256_GRID_100m_E-10000N331400       NaN
done


#  (2) Land Use resampling for 10m

In [2]:
# 1. transformation of land use vector dataset to the 100m Vienna GRID:

# geometry:
import geopandas as gpd

# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_10m_31256.shp")
# 2 LOAD "land uise"
shp = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\landnutzung\REALNUT2020OGD_31256.shp")

# Perform spatial union
union_result = gpd.overlay(shp_100m_GRID, shp, how='union')

# Save the result to a new shapefile
union_result.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\10m_GRID\Real_land_use2020_10m.shp")

print ("done")




C:\Users\Admin\.conda\envs\geospatial_ETC_DI\lib\site-packages\shapely\set_operations.py:77: RuntimeWarning: divide by zero encountered in difference
  return lib.difference(a, b, **kwargs)
C:\Users\Admin\.conda\envs\geospatial_ETC_DI\lib\site-packages\shapely\set_operations.py:77: RuntimeWarning: invalid value encountered in difference
  return lib.difference(a, b, **kwargs)


done


Faster: feature to 10m raster! --> then overly with grid or 10m grid points!

In [ ]:
##Real_land_use2020_100m_test_group_by3
# 1 Load GRID
shp_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\10m_GRID\Real_land_use2020_10m.shp")
out_dem_vector =         r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\10m_GRID\Real_land_use2020_10m_max_land_use_in_cell.shp"

gdf = shp_GRID
#print(gdf.head())


# Calculate the area for each geometry in the GeoDataFrame
gdf['area'] = gdf.geometry.area

# Print the first few rows to check the calculated areas
#print(gdf[['geometry', 'area']].head())


# Group by the attribute field and calculate aggregate statistics
grouped_stats = gdf.groupby(['GRID10mID','NUTZUNG_CO'])['area'].sum()


# Group by 'GRID10mID' and find the index of the row with the maximum area for each group
idx = gdf.groupby('GRID10mID')['area'].idxmax()
# Use the index to select the corresponding 'land_use' value
max_land_use = gdf.loc[idx, ['GRID10mID', 'NUTZUNG_CO']]
max_land_use = max_land_use.rename(columns={'NUTZUNG_CO': 'NUTZ_MAX'})
print(max_land_use.head())
merged_gdf = gdf.merge(max_land_use, on='GRID10mID', how='left')

#print(merged_gdf.head())

# Dissolve the geometries based on the attribute field
dissolved_gdf = merged_gdf.dissolve(by=['GRID10mID','NUTZ_MAX'])



dissolved_gdf.to_file(out_dem_vector)
print ("done")

